# Conda Envrionment install instructions

1. Open anaconda prompt -> cd C:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo
2. Create a folder where you want the env files to be --> mkdir venv
3. conda create --prefix "C:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo\Scene-Recognition\venv" python=3.11.6
4. conda activate "C:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo\Scene-Recognition\venv"
5. conda install -p "C:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo\Scene-Recognition\venv" ipykernel --update-deps --force-reinstall
6. cd Scene-Recognition
7. pip install -r Requirements.txt
8. pip install torch==2.0.1 torchvision==0.15.2 torchaudio==2.0.2 --index-url https://download.pytorch.org/whl/cu118 

# Data Loader Check

In [ ]:
from dataloader import get_data_loader
import torchvision

train_split = 0.75
full_dataset = torchvision.datasets.ImageFolder(root="Data/")
train_size = int(train_split * len(full_dataset))
test_size = int((len(full_dataset) - train_size)/2)
val_size = len(full_dataset) - train_size - test_size

train_size, test_size, val_size

In [ ]:
train_dataloader, test_dataloader, val_dataloader = get_data_loader(batch_size=64,data_dir="Data/", shuffle=True)
for a in train_dataloader:
    print(a[0].shape, a[1].shape)

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_stdlib_context

In [ ]:
import torch

torch.cuda.is_available()

# Model Training

In [3]:
# import packages 
import os 
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np

import torch 
import torch.nn as nn

#import your model here
from log import create_logger
from dataloader import get_data_loader
from models.resnet import resnet18
from models.efficientnet import effnet_s

# Add your models here
models = {'resnet18': resnet18,
         'enet_s':effnet_s,
         }

# RUN DETAILS
run_name = "jly_0206_enets_lr1e-6_bs=64"
model_base = 'enet_s'
num_epochs = 10
bs = 64
lr = 1e-6
random_seed = 42
save_chks = range(num_epochs) # iterable of epochs for which to save the model

device = 'cuda' if torch.cuda.is_available() else ('mps' if torch.backends.mps.is_available() else 'cpu')
if device == 'mps':
    torch.mps.empty_cache()

# set up run dir 
run_dir = os.path.join('saved_models', run_name)
os.makedirs(run_dir, exist_ok = True)
log, logclose = create_logger(log_filename=os.path.join(run_dir, 'train.log'), display = False)
log(f'using device: {device}')
log(f'saving models to: {run_dir}')
log(f'using base model: {model_base}')
log(f'using batch size: {bs}')
log(f'learning rate: {lr}')
log(f'random seed: {random_seed}')

# seed randoms and make deterministic
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
# random.seed(random_seed)
torch.backends.cudnn.enabled=False
torch.backends.cudnn.deterministic=True

# dataloader
train_dataloader, test_dataloader, val_dataloader = get_data_loader(data_dir="Data/",  batch_size=bs, shuffle=True)

# define model 
model = models[model_base]()
model.to(device)

# define optimizer and criterion
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

# training loop
train_loss = []
val_loss = []
test_loss = []
train_metrics = []
val_metrics = []
for epoch in range(num_epochs):
    print(f"epoch: {epoch}")
    log(f'epoch {epoch}')
    #training
    model.train()
    batch_loss = []
    batch_metric = []
    total_imgs = 0
    for i, (_data, _target) in tqdm(enumerate(train_dataloader)): 
        data = _data.to(device)
        target = _target.to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        batch_loss.append(loss.item())
        batch_metric.append(sum(torch.argmax(pred, dim=1)==target).item())
        total_imgs += len(target)
    train_loss.append(sum(np.array(batch_loss)/len(train_dataloader)))
    log(f'\ttrain loss: {train_loss[-1]}')
    train_metrics.append(sum(batch_metric)/total_imgs) #TODO: add metrics
    del data 
    del target
    del pred
    del loss

    # validation
    total_imgs = 0
    batch_metric = []
    with torch.no_grad():
        model.eval()
        batch_loss = []
        for i, (_data, _target) in tqdm(enumerate(val_dataloader)): 
            data = _data.to(device)
            target = _target.to(device)
            pred = model(data)
            loss = criterion(pred, target)
            batch_loss.append(loss.item())
            batch_metric.append(sum(torch.argmax(pred, dim=1)==target).item())
            total_imgs += len(target)
        val_loss.append(sum(np.array(batch_loss)/len(val_dataloader)))
        log(f'\tval loss: {val_loss[-1]}')
        val_metrics.append(sum(batch_metric)/total_imgs) #TODO: add metrics

    if epoch in save_chks: 
        torch.save(model.state_dict(), os.path.join(run_dir, f'{epoch}.chkpt'))

    plt.plot(train_loss, label='train')
    plt.plot(val_loss, label='val')
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend()
    plt.savefig(os.path.join(run_dir, 'loss'))
    plt.close()
    plt.plot(train_metrics, label='train accuracy')
    plt.plot(val_metrics, label='val accuracy')
    plt.xlabel('epoch')
    plt.ylabel('accuracy')
    plt.legend()
    plt.savefig(os.path.join(run_dir, 'accu'))
    plt.close()
    del data 
    del target
    del pred
    del loss

    if device == 'mps':
        torch.mps.empty_cache()


# testing
with torch.no_grad():
    model.eval()

epoch: 0


64it [00:24,  2.66it/s]
14it [00:05,  2.76it/s]


epoch: 1


64it [00:24,  2.63it/s]
14it [00:05,  2.68it/s]


epoch: 2


...

64it [00:23,  2.67it/s]
14it [00:05,  2.78it/s]


epoch: 7


64it [00:24,  2.63it/s]
14it [00:05,  2.74it/s]


epoch: 8


47it [00:17,  2.53it/s]

In [2]:
with torch.no_grad():
    model.eval()
    batch_loss = []
    for i, (_data, _target) in tqdm(enumerate(test_dataloader)): 
        data = _data.to(device)
        target = _target.to(device)
        pred = model(data)
        loss = criterion(pred, target)
        batch_loss.append(loss.item())
        batch_metric.append(sum(torch.argmax(pred, dim=1)==target).item()/len(target))
    test_loss.append(sum(np.array(batch_loss)/len(test_dataloader)))
    log(f'\tval loss: {val_loss[-1]}')
    val_metrics.append(np.mean(batch_metric)) #TODO: add metrics

14it [00:05,  2.69it/s]


In [1]:
from torchvision import models
modelorignal = models.efficientnet_v2_s(pretrained=True)

c:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo\Scene-Recognition\venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\rs659\Desktop\AIPI 590\Group Project 1\Github Repo\Scene-Recognition\venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_V2_S_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_V2_S_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:
from torchsummary import summary

summary(modelorignal.to('cuda'), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 112, 112]             648
       BatchNorm2d-2         [-1, 24, 112, 112]              48
              SiLU-3         [-1, 24, 112, 112]               0
            Conv2d-4         [-1, 24, 112, 112]           5,184
       BatchNorm2d-5         [-1, 24, 112, 112]              48
              SiLU-6         [-1, 24, 112, 112]               0
   StochasticDepth-7         [-1, 24, 112, 112]               0
       FusedMBConv-8         [-1, 24, 112, 112]               0
            Conv2d-9         [-1, 24, 112, 112]           5,184
      BatchNorm2d-10         [-1, 24, 112, 112]              48
             SiLU-11         [-1, 24, 112, 112]               0
  StochasticDepth-12         [-1, 24, 112, 112]               0
      FusedMBConv-13         [-1, 24, 112, 112]               0
           Conv2d-14           [-1, 96,

In [5]:
import torch.nn as nn
from torchvision import models

model_fine_tuned = models.efficientnet_v2_s(weights=models.EfficientNet_V2_S_Weights.DEFAULT)
for param in model_fine_tuned.parameters():
    param.requires_grad = False
model_fine_tuned.avgpool = nn.AdaptiveAvgPool2d(output_size=(1,1))
model_fine_tuned.classifier = nn.Sequential(nn.Dropout(0.2), 
                                    nn.Linear(1280, 5),
                                    nn.Sigmoid(),
                                )

summary(model_fine_tuned.to('cuda'), (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 24, 112, 112]             648
       BatchNorm2d-2         [-1, 24, 112, 112]              48
              SiLU-3         [-1, 24, 112, 112]               0
            Conv2d-4         [-1, 24, 112, 112]           5,184
       BatchNorm2d-5         [-1, 24, 112, 112]              48
              SiLU-6         [-1, 24, 112, 112]               0
   StochasticDepth-7         [-1, 24, 112, 112]               0
       FusedMBConv-8         [-1, 24, 112, 112]               0
            Conv2d-9         [-1, 24, 112, 112]           5,184
      BatchNorm2d-10         [-1, 24, 112, 112]              48
             SiLU-11         [-1, 24, 112, 112]               0
  StochasticDepth-12         [-1, 24, 112, 112]               0
      FusedMBConv-13         [-1, 24, 112, 112]               0
           Conv2d-14           [-1, 96,